# Introduction

The optical design of ACS introduces larger geometric distortion than in other *HST* instruments. From the [ACS Instrument Handbook](http://www.stsci.edu/hst/acs/documents/handbooks/current/cover.html) Section 5.6.4: "The distortions render the pixels, as projected on the sky, trapezoidal in shape and their area varies over the field by about 19% and 3.5% in the WFC and HRC/SBC, respectively."

The [AstroDrizzle](http://drizzlepac.stsci.edu) software transforms ACS images from their distorted frames into the undistorted frame. However, in the event a user wishes to perform photometry on data that have not been processed by AstroDrizzle, a correction must be applied to account for the different sizes of the pixels on the sky across the field of view. A pixel area map (PAM), which is an image where each pixel value describes that pixel's area on the sky relative to the native plate scale, is used for this correction. For ACS/WFC, the plate scale is 0.05 arcseconds/pixel, while for HRC and SBC it is 0.025 arcseconds/pixel.

To transform a distorted FLT/FLC image so that it is suitable for photometry, users must multiply the image by the PAM and divide the result by the exposure time so that the image has units of electrons/second. After this transformation, the information on the [ACS Zeropoints](http://www.stsci.edu/hst/acs/analysis/zeropoints/) page can be used to convert flux measurements into physical units.

<div class="alert alert-block alert-info"><br><b>Note:</b> The distortion corrections applied by both AstroDrizzle and the PAMs include observation-specific corrections (e.g., velocity distortion). For the best results, users are advised to create a PAM for each observation individually.<br></div>

<div class="alert alert-block alert-warning"><br><b>Warning:</b> Before running any code, please be sure you have installed or updated to the latest version of <a href="http://astroconda.readthedocs.io" class="alert-link">AstroConda</a>.<br></div>

### Changes to PAMs in 2018

Previously, the ACS PAMs were provided as static FITS files available for download. However, these files were created in early 2004 and not updated to contain changes to the distortion model. As the model has become more complex and a one-size-fits-all PAM is no longer viable, we now provide Python code that can be used to make the PAMs. Users are advised to use this code, which takes an FLT/FLC FITS file as input, to create PAMs tailored to your data. The code supports all three of the ACS detectors, and it can generate PAMs valid for all dates since the ACS installation in March 2002. Additionally, rather than being an image of the entire detector, the new PAMs have the same shape and position in detector space as the input data, therefore no adjustments to the PAMs need to be made for subarray apertures.

Testing has shown that the new PAMs are within ~0.2% of the 2004 PAMs for images taken in 2018.

#### Example

The code used to make PAMs is packaged with `stsci.skypac` in [AstroConda](http://astroconda.readthedocs.io). Before we begin, we need to download the data, which we can do via `astroquery`:

In [ ]:
from astroquery.mast import Observations
from astropy.io import fits

import os
import glob
import shutil

obs_table = Observations.query_criteria(obs_id='J6ME13QHQ')

Observations.download_products(obs_table['obsid'], mrp_only=False, productSubGroupDescription=['FLT'])

files = glob.glob('mastDownload/HST/*qhq/*.fits')
for f in files:
    try:
        shutil.move(f, '.')
    except:
        os.remove(f.split('/')[-1])
        shutil.move(f, '.')
    
shutil.rmtree('mastDownload')

To create a PAM for the first science extension (WFC2) of this observation, run the following code:

In [ ]:
from stsci.skypac import pamutils

# The pamutils.pam_from_file syntax is (input_file, extension, output_file).
pamutils.pam_from_file('j6me13qhq_flt.fits', 1, 'j6me13qhq_wfc2_pam.fits')

Running this code will produce an output file named "j6mq13qhq_wfc2_pam.fits," which contains the PAM for only the WFC2 science array. For full-frame WFC observations, a second PAM for the WFC1 science array (extension 4) will also need to be created. For subarray WFC data, HRC, and SBC, only one science extension is present. For more information regarding the different extensions, please see the [ACS Data Handbook](http://www.stsci.edu/hst/acs/documents/handbooks/currentDHB/acs_cover.html) Section 2.2